In [3]:
%load_ext autoreload
%autoreload 2

In [76]:
import numpy as np

from models import Sequential
from layers import Dense
from losses import CategoricalCrossentropy, BinaryCrossentropy

from sklearn.metrics import log_loss

loss1 = CategoricalCrossentropy()
loss2 = BinaryCrossentropy()

y_true = np.array([[1, 0],
                    [0, 1]])

y_pred = np.array([[.98, .02],
                    [.3, .7]])

print(loss1(y_true, y_pred))
print(loss2(y_true[:,0,None], y_pred[:,0,None]))
print(log_loss(y_true, y_pred))
print()
print(loss1.derivative(y_true, y_pred)/2)
print(loss2.derivative(y_true[:,0,None], y_pred[:,0,None])/2)
print()
print(y_pred + loss1.derivative(y_true, y_pred)/2)
print(y_pred[:,0,None] + loss2.derivative(y_true[:,0,None], y_pred[:,0,None])/2)

0.18843882562812594
0.18843882562812594
0.18843882562812594

[[-0.25510204 -0.        ]
 [-0.         -0.35714286]]
[[-0.25510204]
 [ 0.35714286]]

[[0.72489796 0.02      ]
 [0.3        0.34285714]]
[[0.72489796]
 [0.65714286]]
